# Applied Seismology, GEOS 626, University of Alaska Fairbanks

- template script for analyzing modes spectra for sumatra
- downloads BHZ and LHZ data needed for hw_sumatraB

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import os
import warnings

from obspy import read
from obspy.core import UTCDateTime

from pysep import Pysep

from lib_seis import station_map_and_table
from lib_seis import sumatra_waveform_screening
from lib_seis import wf_fft

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams['lines.linewidth'] = 1

### Event details
Mw 9.1 2004 Sumatra - Andaman Islands Earthquake \
https://earthquake.usgs.gov/earthquakes/eventpage/official20041226005853450_30

origin time = 2004-12-26 00:58:53 \
latitude    = 3.09 N \
longitude   = 94.26 E \
depth       = 30 km \
magnitude   = 9.1

### PySEP - Python Seismogram Extraction and Processing

We will gather data for this event using the PySEP (https://pysep.readthedocs.io/en/devel/index.html) package. \
This package uses ObsPy (https://docs.obspy.org/) internally to collect and handle seismic data. \
\
Check the following links for details on - \
Data gathering - https://pysep.readthedocs.io/en/devel/autoapi/pysep/pysep/index.html#pysep.pysep.Pysep \
Record section plotting - https://pysep.readthedocs.io/en/devel/autoapi/pysep/recsec/index.html#pysep.recsec.RecordSection \
\
Check the following IRIS webpage for the SEED format seismic channel naming \
https://ds.iris.edu/ds/nodes/dmc/data/formats/seed-channel-naming/

In [ ]:
# download raw LHZ data

download  = dict( client                       = 'IRIS',
                  origin_time                  = UTCDateTime('2004,12,26,00,58,53'),
                  event_latitude               = 3.09,
                  event_longitude              = 94.26,
                  event_depth_km               = 30,
                  event_magnitude              = 9.1,
                  networks                     = 'G,II,IU',
                  stations                     = '*',
                  locations                    = '*',
                  channels                     = 'LHZ',
                  remove_clipped               = False,
                  remove_insufficient_length   = False,
                  remove_response              = False,
                  detrend                      = False,
                  demean                       = False,
                  taper_percentage             = 0,
                  pre_filt                     = None,
                  fill_data_gaps               = 0,
                  gap_fraction                 = 1,
                  resample_freq                = 1,
                  seconds_before_ref           =  0.5 * 24 * 60 * 60,
                  seconds_after_ref            = 10.0 * 24 * 60 * 60,
                  taup_model                   = None,
                  log_level                    = 'CRITICAL',
                  write_files                  = 'sac,inv,event',
                  plot_files                   = 'map',
                  output_dir                   = 'datawf',
                  overwrite                    = False,
                  overwrite_event_tag          = 'sumatra_LHZ_raw')


In [ ]:
# download data

data_dir  = f'{download["output_dir"]}/{download["overwrite_event_tag"]}'
sac_dir   = f'{data_dir}/SAC'
overwrite = f'{download["overwrite"]}'

if (not os.path.isdir(data_dir)) or (overwrite == 'True'):
    ps = Pysep(**download)
    ps.run()
else:
    if os.path.isdir(sac_dir):
        print('SAC directory already exists, proceeding to the next code cell')
    else:
        sys.exit('Data directory exists but with no SAC directory.\n\
            Remove data directory and rerun download.')

In [ ]:
# read downloaded raw LHZ data

st = read(f'{sac_dir}/*.sac', header=None)

print('Details of downloaded waveforms - \n')
print(st.__str__(extended=True))
print()

In [ ]:
# plot waveforms to be rejected

waveforms_to_reject = sumatra_waveform_screening(channel='LHZ')

for i, waveform_id in enumerate(waveforms_to_reject):
    tr = st.select(id=waveform_id)
    plt.figure()
    plt.plot(np.where(tr[0].data==0.00000000, np.nan, tr[0].data))             # replacing filled gaps (0's) with gaps (Nan's)
    plt.xlim(0,907200)
    plt.title(f'{tr[0].id} − starting {tr[0].stats["starttime"]}')

In [ ]:
# create a list of waveforms to retain

retained_waveforms = []                           # stores IDs in the split format ['network','station','location','channel']

for i, tr in enumerate(st):
    if tr.id not in waveforms_to_reject:
        retained_waveforms.append(tr.id.split('.'))

In [ ]:
# plotting and listing station information corresponding to retained waveforms
station_map_and_table(st, st_subset_list=retained_waveforms, event_lat=download['event_latitude'], event_lon=download['event_longitude'])

In [ ]:
# computing and plotting the amplitude spectra of the retained waveforms

if True:

    Nyquist_frequency = download['resample_freq']/2               # resample_freq is provided as input into pysep (see above)

    for i in range(len(st)):

        if st[i].id not in waveforms_to_reject:

            tr = st[i].copy()  
            tr.detrend('constant')
            tr.detrend('linear')
            tr.taper(max_percentage=0.2, type="cosine")

            fft_amp, fft_phase, fft_freq = wf_fft(tr.data, Nyquist_frequency)

            plt.figure()
            plt.plot(fft_freq*1E3,fft_amp)
            plt.xlim(0.2,1)
            #plt.ylim(0,40)
            xmask = np.logical_and(fft_freq*1e3 > .2, fft_freq*1e3 < 1)
            plt.ylim(0,np.max(fft_amp[xmask])*1.1)
            plt.title(f'{tr.id}')
            plt.xlabel(f'Frequency (mHz)')
            plt.ylabel(f'Amplitude (counts s)')
            plt.show()        

In [ ]:
# pick at least 20 waveforms having a useful and clean amplitude spectrum
# pick only one location for a given station

# list of stations with sensors at multiple locations
# network = G  - KIP
# network = II - ALE, ASCN, BORG, DGAR, KDAK, PFO, RPN, WRAB
# network = IU - AFI, ANMO, CASY, COLA, FURI, GUMO, HNR, INCN, KIP, QSPA
#                RAO, RAR, RCBR, SAML, SDV, SNZO, TATO, TEIG, TRIS, TRQA, WAKE

# network | station | location | channel

picked_waveforms = [['G' ,'CAN' ,''  ,'LHZ'],
                    ['II','NNA' ,'00','LHZ'],
                    ['IU','BBSR','00','LHZ'],
                    ['II','SUR' ,'00','LHZ']]

In [ ]:
# plotting and listing station information corresponding to selected waveforms
station_map_and_table(st, st_subset_list=picked_waveforms, event_lat=download['event_latitude'], event_lon=download['event_longitude'])

In [ ]:
# download LHZ data for selected station locations - response removed to acceleration

for i, waveform_id in enumerate(picked_waveforms):

    download  = dict( client                       = 'IRIS',
                      origin_time                  = UTCDateTime("2004,12,26,00,58,53"),
                      event_latitude               = 3.09,
                      event_longitude              = 94.26,
                      event_depth_km               = 30,
                      event_magnitude              = 9.1,
                      networks                     = waveform_id[0],
                      stations                     = waveform_id[1],
                      locations                    = waveform_id[2],
                      channels                     = waveform_id[3],
                      remove_clipped               = False,
                      remove_insufficient_length   = False,
                      remove_response              = True,
                      output_unit                  = 'ACC',
                      detrend                      = True,
                      demean                       = True,
                      taper_percentage             = 0.5,                                       
                      pre_filt                     = 'default',
                      water_level                  = None,
                      fill_data_gaps               = 0,
                      gap_fraction                 = 0.03,
                      resample_freq                = 1,
                      seconds_before_ref           =  0.5 * 24 * 60 * 60,
                      seconds_after_ref            = 10.0 * 24 * 60 * 60,
                      taup_model                   = None,
                      log_level                    = 'DEBUG',
                      write_files                  = 'sac,inv,event',
                      plot_files                   = None,
                      output_dir                   = 'datawf',
                      overwrite                    = True,
                      overwrite_event_tag          = 'sumatra_LHZ_acc')
    
    # download data

    data_dir  = f'{download["output_dir"]}/{download["overwrite_event_tag"]}'
    sac_dir   = f'{data_dir}/SAC'
    overwrite = f'{download["overwrite"]}'

    if (not os.path.isdir(data_dir)) or (overwrite == 'True'):
        ps = Pysep(**download)
        ps.run()
    else:
        if os.path.isdir(sac_dir):
            print('SAC directory already exists, proceeding to the next code cell')
        else:
            sys.exit('Data directory exists but with no SAC directory.\n\
                Remove data directory and rerun download.')

In [ ]:
# read downloaded LHZ acceleration data for the selected station locations

st_acc = read(f'{sac_dir}/*.sac', header=None)

print('Details of downloaded LHZ waveforms with response removed to acceleration for the selected station locations - \n')
print(st_acc.__str__(extended=True))
print()

In [ ]:
# computing and plotting the amplitude spectra of the LHZ acceleration waveforms for the selected station locations

if True:

    Nyquist_frequency = download['resample_freq']/2                # resample_freq is provided as input into pysep (see above)

    for i, tr in enumerate(st_acc):

            tr.detrend('constant')
            tr.detrend('linear')
            tr.taper(max_percentage=0.2, type="cosine")

            fft_amp, fft_phase, fft_freq = wf_fft(tr.data, Nyquist_frequency)

            plt.figure()
            plt.plot(fft_freq*1E3,fft_amp)
            plt.xlim(0.2,1)
            #plt.ylim(0,40)
            xmask = np.logical_and(fft_freq*1e3 > .2, fft_freq*1e3 < 1)
            plt.ylim(0,np.max(fft_amp[xmask])*1.1)
            plt.title(f'{tr.id}')
            plt.xlabel(f'Frequency (mHz)')
            plt.ylabel(f'Amplitude (ms^-2 s)')
            plt.show()        